In [3]:

import numpy as np

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')
from helpers import *
from implementations import *

In [7]:
# loading the data
data_path = '/content/drive/MyDrive/Colab Notebooks/dataset_to_release'
x_train_preclean, x_test_preclean, y_train, train_ids, test_ids = load_csv_data(data_path)

# Data cleaning

In [ ]:
print(x_train_preclean.shape)


(328135, 321)


In [11]:
from collections import defaultdict


def transform_train(feature):
    m = dict()
    for x in feature:
        if x not in m:
            m[x] = len(m)
    f = np.vstack((np.eye(len(m)), np.zeros(len(m))))
    u = f[np.vectorize(lambda key: m.get(key, len(m)))(feature)]
    return u, m


def transform_test(feature, m):
    n_uniq = len(m)
    f = np.vstack((np.eye(n_uniq), np.zeros(n_uniq)))
    ind = np.array([m[k] if k in m else n_uniq for k in feature])
    return f[ind]


def percentageFilled(data):
    return 1 - np.isnan(data).sum() / len(data)


def threshold_col_filter(data, threshold):
    percentage_filled = np.apply_along_axis(percentageFilled, 0, data)
    return percentage_filled > threshold


def non_constant_filter(data):
    return np.logical_not(np.logical_or(np.isnan(np.nanstd(data, 0)), np.nanstd(data, 0) == 0))


cat_threshold = 10

def standardize(x):
    """Standardize the original data set."""
    std = np.nanstd(x, axis=0)
    mean = np.nanmean(x, axis=0)
    return np.nan_to_num((x - np.nanmean(x, axis=0)) / np.nanstd(x, axis=0)), mean, std


def process_train(data):
    n, m = data.shape
    filter = np.logical_and(threshold_col_filter(data, 0.2), non_constant_filter(data))
    categorical_filter = np.apply_along_axis(lambda x: len(set(x)) < cat_threshold, 0, data)
    cat_transform = dict()
    num_transform = dict()
    res = np.empty((n, 0))
    for i in range(m):
        if not filter[i]:
            continue
        if categorical_filter[i]:
            encoded, mp = transform_train(data[:, i])
            cat_transform[i] = mp
            res = np.append(res, encoded, axis=1)
        else:
            x_num_std, mean, std = standardize(data[:, i])
            x_num_std[abs(x_num_std) > 3] = 0
            num_transform[i] = (mean, std)
            res = np.append(res, x_num_std.reshape((n,1)), axis=1)
    return res, filter, categorical_filter, num_transform, cat_transform


def process_test(data, filter, categorical_filter, num_transform, cat_transform):
    n, m = data.shape
    res = np.empty((n, 0))
    for i in range(m):
        if not filter[i]:
            continue
        if categorical_filter[i]:
            res = np.append(res, transform_test(data[:, i], cat_transform[i]), axis=1)
        else:
            mean, std = num_transform[i] # std shouldn't be 0
            res = np.append(res, np.nan_to_num((data[:, i] - mean) / std).reshape((n,1)), axis=1)
    return res







In [12]:
x_train, filter, categorical_filter, num_transform, cat_transform = process_train(x_train_preclean)
print(x_train.shape)

(328135, 428)


In [13]:
x_test = process_test(x_test_preclean, filter, categorical_filter, num_transform, cat_transform)
print(x_test.shape)


(109379, 428)


**Logistic regression with regularization**

In [14]:

def logistic_gradient(y, tx, w):
    return  tx.T.dot(sigmoid(tx.dot(w)) - y)/ len(y)
def compute_gradient_logistic_loss_regularized(y, tx, w, lambda_):
    """Compute the gradient of the regularized logistic regression """
    grad = logistic_gradient(y, tx, w) + lambda_ * w
    return grad

def regularized_log_reg_sgd(y, tx, initial_w, max_iters, gamma,  lambda_ ):
    """Regularized logistic regression using stochastic gradient descent."""
    w = initial_w
    prev_loss = float('inf')

    for n_iter in range(max_iters):
		# Each iteration corresponds to one epoch (num_batches=len(y)) and each batch has size 1
        for batch_y, batch_x in batch_iter(y, tx, 1, num_batches=len(y)):
			# Computing the gradient of the logistic loss with respect to w
            gradient = compute_gradient_logistic_loss_regularized(batch_y, batch_x, w, lambda_)
			# Updating w
            w -= gamma * gradient


        loss = logistic_loss(y, tx, w) + (lambda_ / 2) * np.squeeze(w.T @ w)
        if prev_loss <= loss:
            gamma *= 0.1  # adapt step size
        prev_loss = loss

    return w, loss




In [18]:
# model paramaters (not optimized yet)
initial_w = np.zeros(x_train.shape[1], dtype=np.float64)
max_iters = 100
gamma = 0.01
lambda_ = 0.0001

w, loss = regularized_log_reg_sgd(y_train, x_train, initial_w, max_iters, gamma, lambda_)

In [19]:
print('loss = ', loss)

loss =  0.2211336812499828


In [20]:
def prediction_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix."""
    y_pred = sigmoid(np.dot(data, weights))
    # display(y_pred)
    y_pred[np.where(y_pred >= 0.5)] = 1
    y_pred[np.where(y_pred < 0.5)] = 0
    return y_pred



In [22]:
y_pred = prediction_labels(w, x_train)
temp = y_pred[y_pred != -1]
print("temp", temp.shape)

temp (328135,)


In [23]:
accuracy = (y_pred == y_train).sum() / len(y_train)
print("accuracy", accuracy)

accuracy 0.9146905999055267


**predicting x_test**

In [24]:
y_pred_test = prediction_labels(w, x_test)
y_pred_test.shape

(109379,)

In [25]:
def accuracy(y_pred, y_train):
    return (y_pred == y_train).sum() / len(y_train)
def precision(y_pred, y_train):
    TP = np.sum((y_train==1) & (y_pred==1))
    FP = np.sum((y_train==0) & (y_pred==1))
    return TP/(TP+FP)
def recall(y_pred, y_train):
    recall = np.sum((y_train==1) & (y_pred==1)) / np.sum(y_train==1)
    return recall
def f_score (y_pred, y_train):
    return 2*precision(y_pred, y_train)*recall(y_pred, y_train) / (precision(y_pred, y_train) + recall(y_pred, y_train))

print("accuracy", accuracy(y_pred, y_train))
print("precision", precision(y_pred, y_train))
print("recall", recall(y_pred, y_train))
print("f_score", f_score(y_pred, y_train))


accuracy 0.9146905999055267
precision 0.56360103626943
recall 0.1501639344262295
f_score 0.23714402507153565


In [33]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.

    Args:
        y:      shape=(N,)
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold

    >>> build_k_indices(np.array([1., 2., 3., 4.]), 2, 1)
    array([[3, 2],
           [0, 1]])
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [34]:
def cross_validation(y, x, k_fold, lambda_):
    """
    Separate the training set into k_fold parts, get k_fold sets of w weights, and return the average w
    Args:
        y,
        x,
        k_fold,
        lambda_,

    Return:
        average weight
        losses (loss of each fold)
    """
    k_indices = build_k_indices(y, k_fold, 1)
    losses = []
    weights = []
    for k in range(k_fold):
        test_indices = k_indices[k]
        train_indices = k_indices[~(np.arange(k_indices.shape[0]) == k)].flatten()
        x_train = x[train_indices]
        y_train = y[train_indices]
        x_test = x[test_indices]
        y_test = y[test_indices]

        w, loss = regularized_log_reg_sgd(y_train, x_train, lambda_, np.zeros(x_train.shape[1]), 100, 0.1)
        losses.append(loss)
        weights.append(w)

    return np.mean(weights, axis=0), losses

In [39]:
def get_best_parameters(y, tx, intitial_w, max_iters, gamma, k_fold, lambdas):
    """

    Args:
        y:
        tx:
        intitial_w:
        max_iters:
        gamma:
        lambdas:
        k_fold:

    Returns: best lambda parameter

    """
    seed = 55
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)


    rmse_tr = []
    f1_scores = []

    for lambda_ in lambdas:
        rmsetr_tmp = []
        f1_tmp = []
        for k in range(k_fold):
            # make cross validation return weight
            w, loss = cross_validation(y,tx, k_indices, lambda_)
            rmsetr_tmp.append(loss)
            y_pred = prediction_labels(w, tx)
            f1_tmp.append(f1_score(y_pred, y))

        rmse_tr.append(np.mean(rmsetr_tmp))
        f1_scores.append(np.mean(f1_tmp))
    best_lambda, best_rmse, best_f1  = lambdas[np.argmin(rmse_tr)], np.min(rmse_tr), np.min(f1_scores)
    print( "The best rmse of %.3f is obtained for a lambda of %.5f."%(best_rmse, best_lambda))
    print("The best f1-score of %.3f is obtained for a lambda of %.5f."%(best_f1, best_lambda))
    return best_lambda

In [40]:
best_lambda = get_best_parameters(y_train, x_train, initial_w, max_iters, gamma, k_fold=2, lambdas= np.logspace(-4, 0, 30))

<ipython-input-33-3d072a015080>:17: RuntimeWarning: divide by zero encountered in divide
  interval = int(num_row / k_fold)


TypeError: ignored

# Old Stuff

In [ ]:
subset_idx =[7, 8, 62, 219, 220, 222, 226, 229, 252, 253, 264, 276, 277, 295, 296, 299, 300, 301, 302, 303, 304]

subset = x_train_preclean[:, subset_idx]
subset = standardize(subset)
subset

array([[2.53290043, 2.53290043, 0.28398752, ..., 0.34718589, 0.26522141,
        0.41168639],
       [0.19675462, 0.19675462, 0.24309477, ..., 0.        , 0.        ,
        0.        ],
       [0.10699903, 0.10699903, 4.20921724, ..., 0.34718589, 0.26522141,
        0.41168639],
       ...,
       [0.07267939, 0.07267939, 0.1885711 , ..., 0.34718589, 0.26522141,
        0.41168639],
       [0.18170048, 0.18170048, 0.22037657, ..., 0.28798285, 0.26522141,
        0.36772918],
       [0.97641325, 0.97641325, 0.26581296, ..., 0.34718589, 0.26522141,
        0.41168639]])

### Outliers for Numerical valued

In [10]:
# find the indices of these columns in the data
import matplotlib.pyplot as plt
x = subset
x_std = np.nan_to_num((x - np.nanmean(x, axis=0)) / np.nanstd(x, axis=0))


def box_plot(x, feature_column):
    """
    Illustrating box-plot for a specific feature and printing indices of samples that can be outliers.
    """
    plt.boxplot(x[:, feature_column], showfliers=True)
    plt.title('feature column:' + str(feature_column))
    plt.show()
    index_to_delete = np.where(x[:, feature_column] == max(x[:, feature_column]))[0][0]
    print('sample (potential) outlier index:', index_to_delete)


def box_multi_plot(x):
    """
    Plotting the boxplots for all features
    """
    fig = plt.figure(figsize=(20, 15))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    for featureID in range(x.shape[1]):
        ax = fig.add_subplot(6, 5, featureID + 1)
        ax.boxplot(x[:, featureID], showfliers=True)
        ax.grid(axis='y', alpha=0.75)
        ax.set(title='Feature column:{}'.format(featureID))
def hist_multi_plot(x, color):
    """
    Plotting the histograms of each feature in a design matrix 'tx'.
    """
    fig = plt.figure(figsize=(30, 25))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)
    for featureID in range(x.shape[1]):
        ax = fig.add_subplot(6, 5, featureID + 1)
        ax.hist(x=x[:, featureID], bins='auto', color=color)
        ax.grid(axis='y', alpha=1)
        ax.set(title='Feature column:{}'.format(featureID))
    plt.show()

#hist_multi_plot(x_std, color='blue')

NameError: ignored

In [9]:
#box_multi_plot(x_std)

In [ ]:
# filter out the outliers
def remove_outliers(x_num, threshold = 3):

    x = x_num.copy()
    removed_count = 0
    for col in range(x.shape[1]):
        std = np.std(x[:, col])
        range_ = [-threshold*std, threshold*std]
        # Count outliers for this column
        removed_count_col = np.sum((x[:, col] < range_[0]) | (x[:, col] > range_[1]))
        removed_count += removed_count_col

        # Keep only values within the range
        x[np.logical_or(x[:, col] < range_[0], x[:, col] > range_[1])] = 0

    return x, removed_count


In [ ]:
result, c = remove_outliers(x_std)
print(c / (x_std.shape[0] * x_std.shape[1]))
result

NameError: name 'x_std' is not defined

In [8]:
#box_multi_plot(result)

# Logistic regression *without* regularization

Logisitc regression different than the one in implementations.py. uses stochastic gradient descent. Slight modifications to sigmoid and loss to not make an overflow.

In [ ]:
def sigmoid(t):
    t = np.clip(t, -709, 709)  # Clip to avoid overflow. exp(709) is close to the maximum representable float64
    return np.where(t < 0, np.exp(t)/(1.0 +np.exp(t)) , 1.0 / (1.0 + np.exp(-t)))
def logistic_loss(y, tx, w):
    epsilon = 0.000000001
    y_hat = sigmoid(tx.dot(w))
    y_hat = np.clip(y_hat, epsilon, 1-epsilon)
    loss = - np.average(y*np.log(y_hat) + (1-y)*np.log(1-y_hat))
    return loss

def logistic_gradient(y, tx, w):
    return  tx.T.dot(sigmoid(tx.dot(w)) - y)/ len(y)


def logistic_regression_step(y, tx, initial_w, max_iters, gamma):
    w = initial_w
    prev_loss = float('inf')

    for n_iter in range(max_iters):
       for batch_y, batch_x in batch_iter(y, tx, 1, num_batches=len(tx)):
          gradient = logistic_gradient(batch_y, batch_x, w)
          w = w - gamma * gradient

       loss = logistic_loss(y, tx, w)
       if prev_loss <= loss:
          gamma *= 0.1        # control of the step size
       prev_loss = loss
       #print("SGD iter. {bi}/{ti}: loss={l}, w={}".format(   bi=n_iter, ti=max_iters - 1, l=loss, w=w))

    return w, loss

In [ ]:
initial_w = np.zeros(x_train_std.shape[1], dtype=np.float64) # float64 can be faster and more stable to overflow issues appearantly
max_iters = 100
gamma = 0.1

In [ ]:

w, loss = logistic_regression_step(y_train, x_train_std, initial_w, max_iters, gamma)


In [ ]:
print("loss is ", loss)
print("w is ", w)

loss is  0.22924340476877145
w is  [-3.27714283e+00  5.97358740e-03  1.56328367e-02  1.09390716e-02
 -1.01284227e-02  2.24112448e-02 -2.30569832e-02  0.00000000e+00
  0.00000000e+00 -5.13949071e-01  4.10297682e-02  0.00000000e+00
 -1.70812189e-01  0.00000000e+00 -2.41542597e-02  6.15125785e-03
  1.14175303e-02  5.94134381e-01 -7.39418409e-02 -4.76007255e-03
 -2.10028480e-02 -2.76006599e-03  3.63636231e-02 -3.96158969e-02
  1.01179566e-02 -2.01473154e-01 -8.30796054e-02 -2.80547196e-01
 -2.52487516e-01 -2.50054271e-01 -1.34269832e-01 -1.64759702e-02
 -2.36521036e-02  1.09567745e-02 -5.90979124e-02 -3.19943082e-02
 -2.01536670e-02 -5.30932673e-02 -9.71013368e-02 -5.02039207e-02
 -1.42322480e-02  3.40141487e-02 -1.84255016e-02  2.26267116e-03
 -1.14936679e-01  1.35321426e-01 -5.64887148e-03 -2.71442679e-02
  3.23787539e-02 -4.54569961e-02 -1.57900825e-02 -3.70927422e-02
 -3.69578301e-02 -2.20893986e-02 -4.73067695e-03 -3.25744003e-02
  1.81063129e-02  3.91958186e-03 -1.08893002e-01 -1.378

### Trying to predict x_test

In [ ]:
def prediction_labels(weights, data):
    """Generates class predictions given weights, and a test data matrix."""
    y_pred = sigmoid(np.dot(data, weights))
    # display(y_pred)
    y_pred[np.where(y_pred >= 0.5)] = 1
    y_pred[np.where(y_pred < 0.5)] = 0
    return y_pred



In [ ]:
y_pred = prediction_labels(w, x_train_std)
temp = y_pred[y_pred != -1]
print("temp", temp.shape)

temp (328135,)


In [ ]:
y_train_01 =  np.where(y_train== -1,0,  1) # doesn't change anything ofc

In [ ]:
accuracy = (y_pred == y_train).sum() / len(y_train)
print("accuracy", accuracy)

accuracy 0.9133314032334253


In [ ]:
print("x_train_std", x_train_std.shape, "w shape", w.shape)

x_train_std (328135, 146) w shape (146,)


In [ ]:
# predict on x_test
y_pred_test = prediction_labels(w, x_test_std)
y_pred_test

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
y_pred_test=  np.where(y_pred_test== 0, -1, 1)


109379


In [ ]:
def accuracy(y_pred, y_train):
    return (y_pred == y_train).sum() / len(y_train)
def precision(y_pred, y_train):
    TP = np.sum((y_train==1) & (y_pred==1))
    FP = np.sum((y_train==0) & (y_pred==1))
    return TP/(TP+FP)
def recall(y_pred, y_train):
    recall = np.sum((y_train==1) & (y_pred==1)) / np.sum(y_train==1)
    return recall
def f_score (y_pred, y_train):
    return 2*precision(y_pred, y_train)*recall(y_pred, y_train) / (precision(y_pred, y_train) + recall(y_pred, y_train))

print("accuracy", accuracy(y_pred, y_train))
print("precision", precision(y_pred, y_train))
print("recall", recall(y_pred, y_train))
print("f_score", f_score(y_pred, y_train))


NameError: name 'compute_confusion_matrix_elements' is not defined

## Logistic regression with regularization

In [ ]:
def logistic_gradient(y, tx, w):
    return  tx.T.dot(sigmoid(tx.dot(w)) - y)/ len(y)
def compute_gradient_logistic_loss_regularized(y, tx, w, lambda_):
    """Compute the gradient of the regularized logistic regression """
    grad = logistic_gradient(y, tx, w) + lambda_ * w
    return grad

def regularized_log_reg_sgd(y, tx, initial_w, max_iters, gamma,  lambda_ ):
    """Regularized logistic regression using stochastic gradient descent."""
    w = initial_w
    prev_loss = float('inf')

    for n_iter in range(max_iters):
		# Each iteration corresponds to one epoch (num_batches=len(y)) and each batch has size 1
        for batch_y, batch_x in batch_iter(y, tx, 1, num_batches=len(y)):
			# Computing the gradient of the logistic loss with respect to w
            gradient = compute_gradient_logistic_loss_regularized(batch_y, batch_x, w, lambda_)
			# Updating w
            w -= gamma * gradient


        loss = logistic_loss(y, tx, w) + (lambda_ / 2) * np.squeeze(w.T @ w)
        if prev_loss <= loss:
            gamma *= 0.1  # adapt step size
        prev_loss = loss

    return w, loss




In [ ]:
# model paramaters (not optimized yet)
initial_w = np.zeros(x_train_std.shape[1], dtype=np.float64)
max_iters = 100
gamma = 0.01
lambda_ = 0.0001

w, loss = regularized_log_reg_sgd(y_train, x_train_std, initial_w, max_iters, gamma, lambda_)

In [ ]:
y_pred_reg = prediction_labels(w, x_train_std)


In [ ]:
y_pred_test = prediction_labels(w, x_test)
y_pred_test.shape

ValueError: shapes (109379,201) and (202,) not aligned: 201 (dim 1) != 202 (dim 0)

In [ ]:
accuracy = (y_pred_reg == y_train).sum() / len(y_train)
print("accuracy", accuracy)

accuracy 0.9134228290185442


In [ ]:
# convert 0,1 labels to 1 and -1
y_submit =  np.where(y_pred_test== 0, -1, 1)
print(len(y_submit))
y_submit.sum()
create_csv_submission(test_ids, y_submit, "submition0")


109379


-105523

In [ ]:
create_csv_submission(test_ids, y_submit, "submition0")


In [ ]:

def cross_validation(y, x, k_indices, k, method, **args):
    """
    Completes k-fold cross-validation using the regression method
    """
    # Get k'th subgroup in test, others in train
    tr_indices_set = np.delete(k_indices, k, 0).flatten()
    x_tr = x[tr_indices_set]
    y_tr = y[tr_indices_set]

    te_indices = k_indices[k]
    x_te = x[te_indices]
    y_te = y[te_indices]

    # Apply the regression method
    w, loss = method(y=y_train, tx=x_train_std, **args)

    # Predict outputs with the w
    y_pred = prediction_labels(w, x_te)

    # Calculate accuracy
    #accurancy = compute_accuracy(y_te, predictions)

    #return accuracy

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.

    Args:
        y:      shape=(N,)
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

# feel free to remove the f1 score thing and change parameters
def get_best_parameters(y, tx, intitial_w, max_iters, gamma, lambdas, k_fold):
    """

    Args:
        y:
        tx:
        intitial_w:
        max_iters:
        gamma:
        lambdas:
        k_fold:

    Returns: best lambda parameter

    """
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)


    rmse_tr = []
    f1_scores = []

    for lambda_ in lambdas:
        rmsetr_tmp = []
        f1_tmp = []
        for k in range(k_fold):
            # make cross validation return weight
            w, loss = cross_validation(y,tx, initial_w, max_iters, gamma)
            rmsetr_tmp.append(loss)
            y_pred = prediction_labels(w, tx)
            f1_tmp.append(f1_score(y_pred, y))

        rmse_tr.append(np.mean(rmsetr_tmp))
        f1_scores.append(np.mean(f1_tmp))
    best_lambda, best_rmse, best_f1  = lambdas[np.argmin(rmse_tr)], np.min(rmse_tr), np.min(f1_scores)
    print( "The best rmse of %.3f is obtained for a lambda of %.5f."%(best_rmse, best_lambda))
    print("The best f1-score of %.3f is obtained for a lambda of %.5f."%(best_f1, best_lambda))
    return best_lambda


